# Project 3: Predicting the Success of a Kickstarter Campaign
A supervised learning exercise featuring logistic regression, SVM, KNN, and [Others?]

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from flask_sqlalchemy import SQLAlchemy
import matplotlib.pyplot as plt
import os
import json

In [2]:
import sys
sys.executable

'/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/bin/python'

In [3]:
# env variable at tensorflow1.4 per https://conda.io/docs/user-guide/tasks/manage-environments.html#saving-environment-variables
# and https://vsupalov.com/flask-sqlalchemy-postgres/

def get_env_variable(name):
    try:
        return os.environ[name]
    except KeyError:
        message = "Expected environment variable '{}' not set.".format(name)
        raise Exception(message)

# the values of those depend on your setup
POSTGRES_URL = get_env_variable("POSTGRES_URL")
POSTGRES_USER = get_env_variable("POSTGRES_USER")
POSTGRES_PW = get_env_variable("POSTGRES_PW")
POSTGRES_DB = get_env_variable("POSTGRES_DB")

def to_str(obj):
    return str(obj).split(' ')[0]

def count_list_items(list):
    try:
        return len(list)
    except:
        return 0

def except_zero(fn):
    try:
        return fn
    except:
        return 0

In [4]:
DB_URL = 'postgresql+psycopg2://{user}:{pw}@{url}/{db}'.format(user=POSTGRES_USER,pw=POSTGRES_PW,url=POSTGRES_URL,db=POSTGRES_DB)

In [5]:
engine_var = DB_URL
engine = create_engine(engine_var)

In [6]:
# data at WebRobots https://webrobots.io/kickstarter-datasets/
csv_range = range(1, 49)

df = pd.DataFrame()
for csv in csv_range:
    csv = str(csv).zfill(3)
    path ='data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter{}.csv'.format(csv)
    frame = pd.read_csv(path)

    frame[csv] = csv
    df = df.append(frame) # , ignore_index=True)

In [7]:
print(df.shape)
print(df.columns)
df.head()

(192716, 85)
Index(['001', '002', '003', '004', '005', '006', '007', '008', '009', '010',
       '011', '012', '013', '014', '015', '016', '017', '018', '019', '020',
       '021', '022', '023', '024', '025', '026', '027', '028', '029', '030',
       '031', '032', '033', '034', '035', '036', '037', '038', '039', '040',
       '041', '042', '043', '044', '045', '046', '047', '048', 'backers_count',
       'blurb', 'category', 'converted_pledged_amount', 'country',
       'created_at', 'creator', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'friends', 'fx_rate', 'goal', 'id',
       'is_backing', 'is_starrable', 'is_starred', 'launched_at', 'location',
       'name', 'permissions', 'photo', 'pledged', 'profile', 'slug',
       'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'urls', 'usd_pledged', 'usd_type'],
      dtype='object')


,001,002,003,004,005,006,007,008,009,010,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,stardust-limited-edition-print-launch,https://www.kickstarter.com/discover/categorie...,True,False,successful,1435485613,1.529099,"{""web"":{""project"":""https://www.kickstarter.com...",2061.225115,domestic
1,001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,duality-deck-artist-oracle-cards-and-book,https://www.kickstarter.com/discover/categorie...,True,False,successful,1435784557,1.096800,"{""web"":{""project"":""https://www.kickstarter.com...",5442.323089,domestic
2,001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,rainbow-fantasy-fairy-art-postcard-pixiepocalypse,https://www.kickstarter.com/discover/categorie...,True,False,successful,1435795239,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",5798.000000,domestic
3,001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,day-of-the-dead-thor-calavera-vinyl-stickers,https://www.kickstarter.com/discover/categorie...,True,False,successful,1434418124,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",817.000000,domestic
4,001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,monster-book-project,https://www.kickstarter.com/discover/categorie...,True,False,successful,1435686362,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",2560.000000,domestic


In [8]:
# df['category'].iloc[30000]

In [9]:
df.state.unique()

array(['successful', 'failed', 'live', 'canceled', 'suspended'], dtype=object)

In [10]:
df.groupby('state').nunique()

/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/lib/python3.5/site-packages/pandas/core/groupby.py:3406: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  inc = np.r_[1, val[1:] != val[:-1]]


,001,002,003,004,005,006,007,008,009,010,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
state,,,,,,,,,,,,,,,,,,,,,
canceled,1,1,1,1,1,1,1,1,1,1,...,8693,122,1,2,1,8692,1959,8693,3143,1
failed,1,1,1,1,1,1,1,1,1,1,...,71730,147,1,2,1,70488,6027,71730,20315,2
live,1,1,1,1,1,1,1,1,1,1,...,2960,165,1,2,1,2959,370,3346,1985,2
successful,1,1,1,1,1,1,1,1,1,1,...,91695,168,1,2,1,86230,5548,91718,49498,1
suspended,1,1,1,1,1,1,1,1,1,1,...,592,99,1,1,1,592,191,592,248,1


In [11]:
df = df[['backers_count','blurb', 'category', 'converted_pledged_amount', 'country','created_at', 
         'creator', 'currency', 'currency_symbol','currency_trailing_code', 'current_currency', 
         'deadline','disable_communication', 'friends', 'fx_rate', 'goal', 'id','is_backing', 
         'is_starrable', 'is_starred', 'launched_at', 'location','name', 'permissions', 'photo', 
         'pledged', 'profile', 'slug','source_url', 'staff_pick', 'state', 'state_changed_at',
        'static_usd_rate', 'urls', 'usd_pledged', 'usd_type']]
df = df[(df['state'] == 'successful') | (df['state'] == 'failed')]

In [12]:
df.groupby('state').nunique()

/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/lib/python3.5/site-packages/pandas/core/groupby.py:3406: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  inc = np.r_[1, val[1:] != val[:-1]]


,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,profile,slug,source_url,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
state,,,,,,,,,,,,,,,,,,,,,
failed,482,71393,147,7107,22,71719,72168,14,6,2,...,71730,71730,147,2,1,70488,6027,71730,20315,2
successful,3101,91357,169,28609,22,91668,105474,14,6,2,...,91695,91695,168,2,1,86230,5548,91718,49498,1


In [13]:
df = df = df.drop_duplicates(subset='id')

In [14]:
df.groupby('state').nunique()

/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/lib/python3.5/site-packages/pandas/core/groupby.py:3406: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  inc = np.r_[1, val[1:] != val[:-1]]


,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,profile,slug,source_url,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
state,,,,,,,,,,,,,,,,,,,,,
failed,482,71393,147,7107,22,71719,71577,14,6,2,...,71730,71730,147,2,1,70488,6027,71730,20315,2
successful,3101,91357,169,28609,22,91668,91481,14,6,2,...,91695,91695,158,2,1,86230,5548,91695,49496,1


In [15]:
# import re

df['category_main'] = [json.loads(x)['urls']['web']['discover'][:].split('/')[5] for x in df['category']]
df['category_main'] = df['category_main'].replace({'film%20&%20video': 'film_and_video'})
# rx = re.compile()
df['category_main'].unique()

array(['art', 'comics', 'crafts', 'dance', 'design', 'fashion',
       'film_and_video', 'food', 'games', 'journalism', 'music',
       'photography', 'publishing', 'technology', 'theater'], dtype=object)

In [16]:
df['category_name'] = [json.loads(x)['name'] for x in df['category']]
df['creator_name'] = [json.loads(x)['name'] for x in df['creator']]
df['blurb_len2'] = df['blurb'].str.lower().str.split()
df['blurb_length'] = df['blurb_len2'].apply(count_list_items)

In [17]:
df['created_at'] = pd.to_datetime(df['created_at'],unit='s')
df['launched_at'] = pd.to_datetime(df['launched_at'],unit='s')
df['deadline'] = pd.to_datetime(df['deadline'],unit='s')

df['created'] = pd.to_datetime(df['created_at'])
df['launched'] = pd.to_datetime(df['launched_at'])
df['deadline'] = pd.to_datetime(df['deadline'])

df['created'] = df.created.apply(lambda x: x.date())
df['launched'] = df.launched.apply(lambda x: x.date())
df['deadline'] = df.deadline.apply(lambda x: x.date())

# df.spotlight = df.spotlight.astype(int)
df.staff_pick = df.staff_pick.astype(int)


# print(df['launched'].iloc[0])

# df['created'] = df['created'].apply(lambda x: x.strftime('%Y-%m-%d'))
# df['launched'] = df['launched'].apply(lambda x: x.strftime('%Y-%m-%d'))
# df['deadline'] = df['deadline'].apply(lambda x: x.strftime('%Y-%m-%d'))

df['campaign_length'] = (df['deadline'] - df['launched']).apply(to_str)

In [18]:
print(df.shape)

num_cols = ['backers_count',
            'pledged',
            'usd_pledged',
            'goal',
            'campaign_length',
            'blurb_length',
            'fx_rate']
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')
# df[num_cols] = df[num_cols].fillna(0,inplace=True)
df['usd_goal'] = df['goal'] * (df['usd_pledged']/df['pledged'])
df['pct_goal_achieved'] = round((df['usd_pledged'] / df['usd_goal']),1)
# df['pct_goal_achieved'] = df['pct_goal_achieved'].fillna(0,inplace=True)

df.columns

(163425, 44)


Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'created_at', 'creator', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'friends', 'fx_rate', 'goal', 'id',
       'is_backing', 'is_starrable', 'is_starred', 'launched_at', 'location',
       'name', 'permissions', 'photo', 'pledged', 'profile', 'slug',
       'source_url', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'urls', 'usd_pledged', 'usd_type', 'category_main',
       'category_name', 'creator_name', 'blurb_len2', 'blurb_length',
       'created', 'launched', 'campaign_length', 'usd_goal',
       'pct_goal_achieved'],
      dtype='object')

In [19]:
df = df.sort_values(['deadline'], ascending=[False])
df.shape

(163425, 46)

In [20]:
df = df[['id','name','state','category_main','category_name','backers_count','pct_goal_achieved','usd_pledged','usd_goal','country','currency','campaign_length',
         'deadline','launched','created','staff_pick','creator_name','blurb_length']]
# df = df.dropna()
print(df.shape)
print(df.columns)
df.head()

(163425, 18)
Index(['id', 'name', 'state', 'category_main', 'category_name',
       'backers_count', 'pct_goal_achieved', 'usd_pledged', 'usd_goal',
       'country', 'currency', 'campaign_length', 'deadline', 'launched',
       'created', 'staff_pick', 'creator_name', 'blurb_length'],
      dtype='object')


,id,name,state,category_main,category_name,backers_count,pct_goal_achieved,usd_pledged,usd_goal,country,currency,campaign_length,deadline,launched,created,staff_pick,creator_name,blurb_length
543,1396766240,Ripple Playing Cards - Printed by USPCC,failed,games,Playing Cards,131,0.3,3387.000000,9999.000000,US,USD,38,2018-01-12,2017-12-05,2017-10-08,0,B.Y. Eidelman,16
3545,2065169465,Cotton-Top Pastries,successful,food,Small Batch,99,1.3,9858.000000,7500.000000,US,USD,30,2018-01-12,2017-12-13,2017-12-12,1,Holly Weist,5
130,1647325451,Code Switch,successful,film_and_video,Horror,34,1.5,4611.000000,3000.000000,US,USD,32,2018-01-12,2017-12-11,2017-11-10,0,Alba Roland,23
53,727157486,Rain Dog Farm,failed,food,Farms,49,0.3,4741.000000,18000.000000,US,USD,38,2018-01-12,2017-12-05,2017-11-28,1,Charlie Wainger,9
3146,1756145145,WANGTA: a novel,successful,publishing,Fiction,13,1.0,427.185132,427.185132,CA,CAD,21,2018-01-12,2017-12-22,2017-12-18,0,D. H. de Bruin,22


In [21]:
df.info()
# from pandas.plotting import table

# ax = plt.subplot(111, frame_on=False) # no visible frame
# ax.xaxis.set_visible(False)  # hide the x axis
# ax.yaxis.set_visible(False)  # hide the y axis

# table(ax, df)  # where df is your data frame

# plt.savefig('charts/feature_list.png')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163425 entries, 543 to 980
Data columns (total 18 columns):
id                   163425 non-null int64
name                 163424 non-null object
state                163425 non-null object
category_main        163425 non-null object
category_name        163425 non-null object
backers_count        163425 non-null int64
pct_goal_achieved    147802 non-null float64
usd_pledged          163425 non-null float64
usd_goal             147802 non-null float64
country              163425 non-null object
currency             163425 non-null object
campaign_length      163425 non-null int64
deadline             163425 non-null object
launched             163425 non-null object
created              163425 non-null object
staff_pick           163425 non-null int64
creator_name         163425 non-null object
blurb_length         163425 non-null int64
dtypes: float64(3), int64(5), object(10)
memory usage: 23.7+ MB


In [22]:
df.tail()

,id,name,state,category_main,category_name,backers_count,pct_goal_achieved,usd_pledged,usd_goal,country,currency,campaign_length,deadline,launched,created,staff_pick,creator_name,blurb_length
1785,1578671837,Support casting my sculpture in bronze,failed,art,Sculpture,1,0.0,25.0,2000.0,US,USD,30,2009-05-31,2009-05-01,2009-05-01,0,C.K. Sample,18
2352,266044220,Help me write my second novel.,successful,publishing,Fiction,18,1.1,563.0,500.0,US,USD,30,2009-05-29,2009-04-29,2009-04-29,1,Brendan Mackie,24
2373,199916122,Mr. Squiggles,failed,art,Illustration,0,NaN,0.0,NaN,US,USD,10,2009-05-22,2009-05-12,2009-05-12,0,C.K. Sample,22
0,2089078683,New York Makes a Book!!,successful,journalism,Journalism,110,1.1,3329.0,3000.0,US,USD,18,2009-05-16,2009-04-28,2009-04-27,1,We Make a Book,22
980,1703704063,drawing for dollars,successful,art,Illustration,3,1.8,35.0,20.0,US,USD,9,2009-05-03,2009-04-24,2009-04-24,1,darkpony,26


In [23]:
df.staff_pick.sum()

22017

In [24]:
# df.state.sum()

In [25]:
df.shape

(163425, 18)

In [26]:
df.columns

Index(['id', 'name', 'state', 'category_main', 'category_name',
       'backers_count', 'pct_goal_achieved', 'usd_pledged', 'usd_goal',
       'country', 'currency', 'campaign_length', 'deadline', 'launched',
       'created', 'staff_pick', 'creator_name', 'blurb_length'],
      dtype='object')

In [27]:
df[df['state'] == 'failed'].count()

id                   71730
name                 71729
state                71730
category_main        71730
category_name        71730
backers_count        71730
pct_goal_achieved    56107
usd_pledged          71730
usd_goal             56107
country              71730
currency             71730
campaign_length      71730
deadline             71730
launched             71730
created              71730
staff_pick           71730
creator_name         71730
blurb_length         71730
dtype: int64

In [28]:
df[df['state'] == 'successful'].count()

id                   91695
name                 91695
state                91695
category_main        91695
category_name        91695
backers_count        91695
pct_goal_achieved    91695
usd_pledged          91695
usd_goal             91695
country              91695
currency             91695
campaign_length      91695
deadline             91695
launched             91695
created              91695
staff_pick           91695
creator_name         91695
blurb_length         91695
dtype: int64

In [29]:
df.head()

,id,name,state,category_main,category_name,backers_count,pct_goal_achieved,usd_pledged,usd_goal,country,currency,campaign_length,deadline,launched,created,staff_pick,creator_name,blurb_length
543,1396766240,Ripple Playing Cards - Printed by USPCC,failed,games,Playing Cards,131,0.3,3387.000000,9999.000000,US,USD,38,2018-01-12,2017-12-05,2017-10-08,0,B.Y. Eidelman,16
3545,2065169465,Cotton-Top Pastries,successful,food,Small Batch,99,1.3,9858.000000,7500.000000,US,USD,30,2018-01-12,2017-12-13,2017-12-12,1,Holly Weist,5
130,1647325451,Code Switch,successful,film_and_video,Horror,34,1.5,4611.000000,3000.000000,US,USD,32,2018-01-12,2017-12-11,2017-11-10,0,Alba Roland,23
53,727157486,Rain Dog Farm,failed,food,Farms,49,0.3,4741.000000,18000.000000,US,USD,38,2018-01-12,2017-12-05,2017-11-28,1,Charlie Wainger,9
3146,1756145145,WANGTA: a novel,successful,publishing,Fiction,13,1.0,427.185132,427.185132,CA,CAD,21,2018-01-12,2017-12-22,2017-12-18,0,D. H. de Bruin,22


In [30]:
df.groupby('state').nunique()

,id,name,state,category_main,category_name,backers_count,pct_goal_achieved,usd_pledged,usd_goal,country,currency,campaign_length,deadline,launched,created,staff_pick,creator_name,blurb_length
state,,,,,,,,,,,,,,,,,,
failed,71730,71593,1,15,137,482,12,20315,15546,22,14,92,2975,3013,3011,2,65012,35
successful,91695,91531,1,15,159,3101,729,49496,16718,22,14,92,3077,3122,3132,2,76695,35


In [32]:
df.to_pickle('data/kickstarter_data_ds2.pkl')
# try:
#     df.to_sql("kickstarter_data_ds2", engine, chunksize=20000, if_exists='replace')
# except Exception as e:
#     print(e)
#     pass

# Temporary fix:

# df.to_csv('data/kickstarter_data.csv')